## Evaluation Metrics


In order to calculate the evaluation metrics quickly, we decided to do it in Bigquery.

In [1]:
import pandas as pd
import pandas_gbq

In [6]:
def get_metrics(table_id, k, project_id, minimum_orders=2, maximum_orders=1000, is_most_popular=False, is_context_model=False):
    
    if is_most_popular:
        comm1 = "--"
        comm2 = ""
    else:
        comm1 = ""
        comm2 = "--"
        
    if is_context_model:
        comm3 = "--"
        comm4 = ""
    else:
        comm3 = ""
        comm4 = "--"   

    query = f"""
    
    DECLARE minimum_products INT64;
    DECLARE minimum_orders INT64;
    DECLARE maximum_orders INT64;
    DECLARE k INT64;

    SET minimum_products = 5;
    SET minimum_orders = {minimum_orders};
    SET maximum_orders = {maximum_orders};
    SET k = {k};

    WITH products_by_user AS (
        SELECT
            uo.user_id
          , COUNT(DISTINCT gtin) as cant_products
          , COUNT(DISTINCT uo.order_id) as cant_orders
          , MAX(uo.order_id) as last_order_id
        FROM 
          `peya-food-and-groceries.user_fiorella_dirosario.order_sep2020_sep2021` as uo
        JOIN 
          `peya-food-and-groceries.user_fiorella_dirosario.order_details_sep2020_sep2021` as od
        ON 
          uo.order_id = od.order_id  
        WHERE 
          uo.user_id IS NOT NULL
          AND od.gtin IS NOT NULL
          AND od.has_gtin = 1
        GROUP BY 1
    )

    , test AS (
    SELECT DISTINCT
        CAST(uo.user_id AS STRING) AS user_id
      , CAST(od.gtin AS STRING) AS product_id
      --, uo.order_id
      --, uo.timestamp
    FROM 
      `peya-food-and-groceries.user_fiorella_dirosario.order_sep2020_sep2021` as uo
    JOIN 
      `peya-food-and-groceries.user_fiorella_dirosario.order_details_sep2020_sep2021` as od
    ON
      uo.order_id = od.order_id
    LEFT JOIN 
      products_by_user pbu 
    ON 
      pbu.user_id = uo.user_id
    WHERE 
      uo.user_id IS NOT NULL
      AND od.gtin IS NOT NULL
      AND od.has_gtin = 1
      AND cant_products >= minimum_products
      AND cant_orders >= minimum_orders
      AND cant_orders <= maximum_orders
      AND uo.order_id = pbu.last_order_id 
    )

    , predictions AS 

    (
        {comm3}SELECT * FROM `{table_id}` WHERE rank < k
        {comm4}SELECT CAST(user_id AS STRING) as user_id, gtin as product_id, rank FROM `{table_id}` WHERE rank < k
    )


    , cross_join AS (
    SELECT
    t.user_id
    , p.rank
    , p.product_id as product_predicted
    , t.product_id  as product_test
    , CASE WHEN t.product_id = p.product_id THEN 1 ELSE 0 END as hit_rate
    , CASE WHEN t.product_id = p.product_id THEN 1 / (p.rank + 1) ELSE 0 END as map_at_k
    FROM test t 
    {comm1}LEFT JOIN predictions p ON p.user_id = t.user_id
    {comm2}CROSS JOIN predictions p
    WHERE p.rank < k
    ORDER BY 1, 4, 2
    )

    , metric_by_user AS (
    SELECT
    user_id
    , SUM(hit_rate) as hit_rate
    , SUM(map_at_k) as map_at_k
    FROM cross_join 
    GROUP BY 1

    )

    SELECT
    COUNT(DISTINCT user_id) as users
    , AVG(hit_rate) as hit
    , AVG(map_at_k) as precision
    FROM metric_by_user
    
    """
    
    df = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

    return df


### Random Predictions

In [5]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_random', k=5, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.55rows/s]


,hit,precision
0,0.000875,0.000395


In [15]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_random', k=10, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.44rows/s]


,hit,precision
0,0.001854,0.000521


### k-popular Products

In [13]:
get_metrics(table_id='peya-food-and-groceries.user_fiorella_dirosario.popular_products_train', k=5, project_id='peya-growth-and-onboarding', is_most_popular=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.92rows/s]


,hit,precision
0,0.083081,0.045322


In [14]:
get_metrics(table_id='peya-food-and-groceries.user_fiorella_dirosario.popular_products_train', k=10, project_id='peya-growth-and-onboarding', is_most_popular=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.83rows/s]


,hit,precision
0,0.136766,0.052362


In [29]:
get_metrics(table_id='peya-food-and-groceries.user_fiorella_dirosario.popular_products_train', k=50, project_id='peya-growth-and-onboarding', is_most_popular=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.52rows/s]


,hit,precision
0,0.465319,0.066248


In [30]:
get_metrics(table_id='peya-food-and-groceries.user_fiorella_dirosario.popular_products_train', k=100, project_id='peya-growth-and-onboarding', is_most_popular=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.44rows/s]


,hit,precision
0,0.724531,0.069911


In [17]:
get_metrics(table_id='peya-food-and-groceries.user_fiorella_dirosario.popular_products_train', minimum_orders=2, maximum_orders=5, k=10, project_id='peya-growth-and-onboarding', is_most_popular=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.67rows/s]


,users,hit,precision
0,228927,0.128325,0.051607


In [18]:
get_metrics(table_id='peya-food-and-groceries.user_fiorella_dirosario.popular_products_train', minimum_orders=6, maximum_orders=10, k=10, project_id='peya-growth-and-onboarding', is_most_popular=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.81rows/s]


,users,hit,precision
0,60176,0.142249,0.052337


In [19]:
get_metrics(table_id='peya-food-and-groceries.user_fiorella_dirosario.popular_products_train', minimum_orders=11, maximum_orders=10000, k=10, project_id='peya-growth-and-onboarding', is_most_popular=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.64rows/s]


,users,hit,precision
0,67478,0.160512,0.054946


### Baseline Model

In [16]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_baseline', k=5, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.47rows/s]


,hit,precision
0,0.11113,0.058682


In [18]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_baseline', k=10, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.38rows/s]


,hit,precision
0,0.174757,0.067039


In [14]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_baseline', k=10, minimum_orders=2, maximum_orders=5, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.91rows/s]


,users,hit,precision
0,228927,0.189768,0.075524


In [15]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_baseline', k=10, minimum_orders=6, maximum_orders=10, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.84rows/s]


,users,hit,precision
0,60176,0.168223,0.059901


In [16]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_baseline', k=10, minimum_orders=11, maximum_orders=10000, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.61rows/s]


,users,hit,precision
0,67478,0.129657,0.044617


In [14]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_baseline', k=10, minimum_orders=2, maximum_orders=5, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.91rows/s]


,users,hit,precision
0,228927,0.189768,0.075524


In [15]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_baseline', k=10, minimum_orders=6, maximum_orders=10, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.84rows/s]


,users,hit,precision
0,60176,0.168223,0.059901


In [16]:
get_metrics(table_id='peya-growth-and-onboarding.user_patricio_woodley.user_recommendation_baseline', k=10, minimum_orders=11, maximum_orders=10000, project_id='peya-growth-and-onboarding', is_most_popular=False)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.61rows/s]


,users,hit,precision
0,67478,0.129657,0.044617


### Context Model

In [23]:
get_metrics(table_id="peya-food-and-groceries.user_fiorella_dirosario.recommendation_context_model", k=5, project_id='peya-growth-and-onboarding', is_context_model=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.46rows/s]


,hit,precision
0,0.038249,0.019447


In [24]:
get_metrics(table_id="peya-food-and-groceries.user_fiorella_dirosario.recommendation_context_model", k=10, project_id='peya-growth-and-onboarding', is_context_model=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.09rows/s]


,hit,precision
0,0.064157,0.022833


In [11]:
get_metrics(table_id="peya-food-and-groceries.user_fiorella_dirosario.recommendation_context_model", minimum_orders=2, maximum_orders=5, k=10, project_id='peya-growth-and-onboarding', is_context_model=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.84rows/s]


,users,hit,precision
0,228927,0.059412,0.021563


In [12]:
get_metrics(table_id="peya-food-and-groceries.user_fiorella_dirosario.recommendation_context_model", minimum_orders=6, maximum_orders=10, k=10, project_id='peya-growth-and-onboarding', is_context_model=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.86rows/s]


,users,hit,precision
0,60176,0.069114,0.023902


In [13]:
get_metrics(table_id="peya-food-and-groceries.user_fiorella_dirosario.recommendation_context_model", minimum_orders=11, maximum_orders=10000, k=10, project_id='peya-growth-and-onboarding', is_context_model=True)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.83rows/s]


,users,hit,precision
0,67478,0.075832,0.026187
